In [1]:
library(tidyverse)
library(targets)
library(tarchetypes)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [12]:
combination_recipe = qs::qread("combination_recipe_bmp1.qs")

In [13]:
combination_recipe = combination_recipe %>% select(output_name, output_suffix, day, contrast_name)
combination_recipe

output_name,output_suffix,day,contrast_name
<chr>,<chr>,<chr>,<chr>
combined_da_results_all.BL6d5d14,all.BL6d5d14,all,BL6d5d14
combined_da_results_all.BMP1_LDNvVehicle,all.BMP1_LDNvVehicle,all,BMP1_LDNvVehicle
combined_da_results_all.BMP1_Vehicle_vs_FGF1_Veh_PF,all.BMP1_Vehicle_vs_FGF1_Veh_PF,all,BMP1_Vehicle_vs_FGF1_Veh_PF
combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14
combined_da_results_all.obob14vBL6,all.obob14vBL6,all,obob14vBL6
combined_da_results_all.obob5v5,all.obob5v5,all,obob5v5
combined_da_results_all.obob5vBL6,all.obob5vBL6,all,obob5vBL6
combined_da_results_all.obobBL6,all.obobBL6,all,obobBL6
combined_da_results_all.obobBL6d14,all.obobBL6d14,all,obobBL6d14


In [14]:
fgf1_contrasts = combination_recipe %>%
    filter(contrast_name %in% c("obob5v5", "obob14v14"))
colnames(fgf1_contrasts) = paste0(colnames(fgf1_contrasts), '.fgf1')
fgf1_contrasts

output_name.fgf1,output_suffix.fgf1,day.fgf1,contrast_name.fgf1
<chr>,<chr>,<chr>,<chr>
combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14
combined_da_results_all.obob5v5,all.obob5v5,all,obob5v5
combined_da_results_Day5.obob5v5,Day5.obob5v5,Day5,obob5v5


In [15]:
bl6_contrasts = combination_recipe %>%
    filter(contrast_name %in% c("obobBL6", "obobBL6d5", "obobBL6d14", "obob5vBL6", "obob14vBL6"))
colnames(bl6_contrasts) = paste0(colnames(bl6_contrasts), '.BL6')
bl6_contrasts

output_name.BL6,output_suffix.BL6,day.BL6,contrast_name.BL6
<chr>,<chr>,<chr>,<chr>
combined_da_results_all.obob14vBL6,all.obob14vBL6,all,obob14vBL6
combined_da_results_all.obob5vBL6,all.obob5vBL6,all,obob5vBL6
combined_da_results_all.obobBL6,all.obobBL6,all,obobBL6
combined_da_results_all.obobBL6d14,all.obobBL6d14,all,obobBL6d14
combined_da_results_all.obobBL6d5,all.obobBL6d5,all,obobBL6d5
combined_da_results_Day5.obobBL6d5,Day5.obobBL6d5,Day5,obobBL6d5


In [16]:
restored_recipe = tidyr::crossing(fgf1_contrasts, bl6_contrasts) %>%
filter(day.fgf1 == day.BL6) %>%
mutate(restored_suffix= paste0(output_suffix.fgf1, '__v__', output_suffix.BL6)) %>%
mutate(output_obj.BL6 = rlang::syms(output_name.BL6)) %>%
mutate(output_obj.fgf1 = rlang::syms(output_name.fgf1)) %>%
relocate(restored_suffix, output_obj.fgf1, output_obj.BL6)


qs::qsave(restored_recipe, "restored_recipe_bmp1.qs")
restored_recipe

restored_suffix,output_obj.fgf1,output_obj.BL6,output_name.fgf1,output_suffix.fgf1,day.fgf1,contrast_name.fgf1,output_name.BL6,output_suffix.BL6,day.BL6,contrast_name.BL6
<chr>,<list>,<list>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
all.obob14v14__v__all.obob14vBL6,combined_da_results_all.obob14v14,combined_da_results_all.obob14vBL6,combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14,combined_da_results_all.obob14vBL6,all.obob14vBL6,all,obob14vBL6
all.obob14v14__v__all.obob5vBL6,combined_da_results_all.obob14v14,combined_da_results_all.obob5vBL6,combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14,combined_da_results_all.obob5vBL6,all.obob5vBL6,all,obob5vBL6
all.obob14v14__v__all.obobBL6,combined_da_results_all.obob14v14,combined_da_results_all.obobBL6,combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14,combined_da_results_all.obobBL6,all.obobBL6,all,obobBL6
all.obob14v14__v__all.obobBL6d14,combined_da_results_all.obob14v14,combined_da_results_all.obobBL6d14,combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14,combined_da_results_all.obobBL6d14,all.obobBL6d14,all,obobBL6d14
all.obob14v14__v__all.obobBL6d5,combined_da_results_all.obob14v14,combined_da_results_all.obobBL6d5,combined_da_results_all.obob14v14,all.obob14v14,all,obob14v14,combined_da_results_all.obobBL6d5,all.obobBL6d5,all,obobBL6d5
all.obob5v5__v__all.obob14vBL6,combined_da_results_all.obob5v5,combined_da_results_all.obob14vBL6,combined_da_results_all.obob5v5,all.obob5v5,all,obob5v5,combined_da_results_all.obob14vBL6,all.obob14vBL6,all,obob14vBL6
all.obob5v5__v__all.obob5vBL6,combined_da_results_all.obob5v5,combined_da_results_all.obob5vBL6,combined_da_results_all.obob5v5,all.obob5v5,all,obob5v5,combined_da_results_all.obob5vBL6,all.obob5vBL6,all,obob5vBL6
all.obob5v5__v__all.obobBL6,combined_da_results_all.obob5v5,combined_da_results_all.obobBL6,combined_da_results_all.obob5v5,all.obob5v5,all,obob5v5,combined_da_results_all.obobBL6,all.obobBL6,all,obobBL6
all.obob5v5__v__all.obobBL6d14,combined_da_results_all.obob5v5,combined_da_results_all.obobBL6d14,combined_da_results_all.obob5v5,all.obob5v5,all,obob5v5,combined_da_results_all.obobBL6d14,all.obobBL6d14,all,obobBL6d14
